<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_VAQ_VQE_mkbahk_20251208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 먼저 기존 qiskit 관련 패키지 모두 제거 (깨끗한 환경 권장)
!pip uninstall -y qiskit qiskit-nature qiskit-aer qiskit-algorithms qiskit-terra pyscf

# Python 3.9 ~ 3.11 환경에서 아래 순서대로 설치
!pip install qiskit==1.2.4
!pip install qiskit-aer==0.15.1
!pip install qiskit-algorithms==0.3.1
!pip install qiskit-nature[pyscf]==0.7.2
!pip install pyscf==2.6.0

Found existing installation: qiskit 1.2.4
Uninstalling qiskit-1.2.4:
  Successfully uninstalled qiskit-1.2.4
  Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached qiskit_aer-0.15.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.0 kB)
Using cached qiskit_aer-0.15.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
Using cached qiskit_algorithms-0.3.1-py3-none-any.whl (310 kB)
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached pyscf-2.11.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)
Using cached pyscf-2.11.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (51.3 MB)
  Using cached pyscf-2.6.0-

In [2]:
"""
VQE (Variational Quantum Eigensolver) 구현
수소 분자(H2)의 바닥 상태 에너지 계산
"""

'\nVQE (Variational Quantum Eigensolver) 구현\n수소 분자(H2)의 바닥 상태 에너지 계산\n'

In [3]:
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, COBYLA
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_aer import AerSimulator

In [4]:
print("=" * 60)
print("VQE를 이용한 분자 에너지 계산")
print("=" * 60)

VQE를 이용한 분자 에너지 계산


In [5]:
# 1. 분자 정의 (H2 분자)
print("\n[1단계] 분자 구조 정의")
print("-" * 60)

# 수소 분자 정의 (원자간 거리: 0.735 Angstrom)
molecule = "H 0.0 0.0 0.0; H 0.0 0.0 0.735"
driver = PySCFDriver(
    atom=molecule,
    basis="sto3g",  # 기저 함수
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

print(f"분자: H2 (수소 분자)")
print(f"원자간 거리: 0.735 Å")
print(f"기저 함수: STO-3G")


[1단계] 분자 구조 정의
------------------------------------------------------------
분자: H2 (수소 분자)
원자간 거리: 0.735 Å
기저 함수: STO-3G


In [6]:
# 2. 전자 구조 문제 설정
print("\n[2단계] 전자 구조 문제 설정")
print("-" * 60)

problem = driver.run()
print(f"분자 궤도 개수: {problem.num_spatial_orbitals}")
print(f"전자 개수: {problem.num_particles}")

# Hamiltonian 얻기
hamiltonian = problem.hamiltonian.second_q_op()


[2단계] 전자 구조 문제 설정
------------------------------------------------------------
분자 궤도 개수: 2
전자 개수: (1, 1)


In [7]:
# 3. Qubit Mapping (Jordan-Wigner 변환)
print("\n[3단계] Qubit Mapping")
print("-" * 60)

mapper = JordanWignerMapper()
qubit_op = mapper.map(hamiltonian)

print(f"필요한 큐비트 개수: {qubit_op.num_qubits}")
print(f"Pauli 항의 개수: {len(qubit_op)}")


[3단계] Qubit Mapping
------------------------------------------------------------
필요한 큐비트 개수: 4
Pauli 항의 개수: 15


In [8]:
# 4. Ansatz 정의 (양자 회로)
print("\n[4단계] Ansatz (Variational Form) 정의")
print("-" * 60)

num_qubits = qubit_op.num_qubits

# TwoLocal ansatz 사용
ansatz = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks=['ry', 'rz'],
    entanglement_blocks='cz',
    entanglement='linear',
    reps=2,
    insert_barriers=True
)

print(f"Ansatz 타입: TwoLocal")
print(f"회전 게이트: RY, RZ")
print(f"얽힘 게이트: CZ")
print(f"반복 횟수: 2")
print(f"파라미터 개수: {ansatz.num_parameters}")


[4단계] Ansatz (Variational Form) 정의
------------------------------------------------------------
Ansatz 타입: TwoLocal
회전 게이트: RY, RZ
얽힘 게이트: CZ
반복 횟수: 2
파라미터 개수: 24


In [9]:
# 5. 시뮬레이터 설정
print("\n[5단계] 양자 시뮬레이터 설정")
print("-" * 60)

backend = AerSimulator()
print(f"백엔드: {backend.name}")

# Estimator 설정 (Qiskit Primitives)
estimator = Estimator()


[5단계] 양자 시뮬레이터 설정
------------------------------------------------------------
백엔드: aer_simulator


/tmp/ipython-input-2725445818.py:9: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [10]:
# 6. Optimizer 설정
print("\n[6단계] Classical Optimizer 설정")
print("-" * 60)

optimizer = SLSQP(maxiter=100)
print(f"Optimizer: SLSQP")
print(f"최대 반복 횟수: 100")


[6단계] Classical Optimizer 설정
------------------------------------------------------------
Optimizer: SLSQP
최대 반복 횟수: 100


In [11]:
# 7. VQE 실행
print("\n[7단계] VQE 알고리즘 실행")
print("-" * 60)
print("최적화 진행 중...")

# 초기 파라미터 설정
initial_point = np.random.random(ansatz.num_parameters)

# VQE 인스턴스 생성
vqe = VQE(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point
)

# VQE 실행
result = vqe.compute_minimum_eigenvalue(qubit_op)


[7단계] VQE 알고리즘 실행
------------------------------------------------------------
최적화 진행 중...


In [ ]:
# 8. 결과 출력
print("\n" + "=" * 60)
print("VQE 결과")
print("=" * 60)

print(f"\n최적 에너지 (VQE): {result.eigenvalue.real:.6f} Hartree")
print(f"최적 파라미터 개수: {len(result.optimal_point)}")
print(f"Optimizer 평가 횟수: {result.cost_function_evals}")

In [ ]:
# 9. 정확한 해와 비교 (Classical 계산)
print("\n[비교] Classical Exact Eigensolver")
print("-" * 60)

from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
exact_result = numpy_solver.compute_minimum_eigenvalue(qubit_op)

print(f"정확한 에너지: {exact_result.eigenvalue.real:.6f} Hartree")

# 오차 계산
error = abs(result.eigenvalue.real - exact_result.eigenvalue.real)
print(f"\n에너지 오차: {error:.6f} Hartree")
print(f"상대 오차: {(error / abs(exact_result.eigenvalue.real)) * 100:.4f}%")

In [ ]:
# 10. 추가 정보
print("\n" + "=" * 60)
print("추가 정보")
print("=" * 60)

# 에너지를 다른 단위로 변환 (Hartree to eV)
hartree_to_ev = 27.211386245988
vqe_energy_ev = result.eigenvalue.real * hartree_to_ev
exact_energy_ev = exact_result.eigenvalue.real * hartree_to_ev

print(f"\nVQE 에너지: {vqe_energy_ev:.6f} eV")
print(f"정확한 에너지: {exact_energy_ev:.6f} eV")

print("\n" + "=" * 60)
print("계산 완료!")
print("=" * 60)